# 03_TRAIN_CONTINUAL — Entrenamiento incremental (continual) con Udacity → SNN

__Objetivo__. Entrenar y evaluar un pipeline continual (2 tareas: `circuito1`→`circuito2`) con opciones de:

- Codificación temporal en runtime (CPU/GPU) o spikes offline (H5).
- Balanceo offline (CSV `train_balanced.csv`) o balanceo online (sampler).
- Presets (`fast`|`std`|`accurate`) + métodos (`naive`|`ewc`|`rehearsal`|`rehearsal+ewc`).

Requisitos previos.

Haber ejecutado la __preparación__ de datos:
`python tools/prep_udacity.py --root . --runs circuito1 circuito2 --use-left-right --steer-shift 0.2 --bins 21 --train 0.70 --val 0.15 --target-per-bin auto --cap-per-bin 12000 --seed 42`

(Opcional) Generar __H5__ si vas a usar spikes offline:
`python tools/encode_tasks.py --tasks-file data/processed/tasks_balanced.json --encoder rate --T 10 --gain 0.5 --w 200 --h 66`

__Imports y setup de entorno__

In [1]:
# =============================================================================
# Imports y setup de entorno (threads, paths, dispositivo)
# =============================================================================
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, torch

# Raíz del repo y sys.path
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Librerías del proyecto
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model, default_tfm_for_model

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ajustes de rendimiento (opcional)
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")

print("Device:", device)


Device: cuda


__Configuración global (modelo, codificación y loaders)__

In [2]:
# =============================================================================
# Config global: modelo, codificación temporal y loaders
# =============================================================================
SEED = 42

# Modelo: "snn_vision" | "pilotnet_ann" | "pilotnet_snn"
MODEL_NAME = "pilotnet_snn"

# Transform por defecto según el modelo
# (PilotNet suele usar 200x66; snn_vision, 160x80; ambas en gris por defecto)
tfm = default_tfm_for_model(MODEL_NAME, to_gray=True)

# --- Flags de ejecución -----------------------------------------------
# 1) ¿Usar spikes guardados en H5? (False = codificar en runtime)
USE_OFFLINE_SPIKES = False

# 2) ¿Balanceo offline (tasks_balanced.json) o tasks.json normal?
USE_OFFLINE_BALANCED = True

# 3) ¿Balanceo online (sampler) durante el entrenamiento?
#    Solo tiene sentido si el CSV de train NO es train_balanced.csv.
USE_ONLINE_BALANCING = False

# Nombre unificado (antes tenías GPU_ENCODE): True = codificación en runtime
RUNTIME_ENCODE = (not USE_OFFLINE_SPIKES)

# Parámetros de DataLoader
SAFE_MODE     = False
NUM_WORKERS   = 12
PREFETCH      = 2
PIN_MEMORY    = True
PERSISTENT    = True

# Augmentación (solo train)
AUG_CFG_LIGHT = AugmentConfig(prob_hflip=0.5, brightness=None,           gamma=None,           noise_std=0.0)
AUG_CFG_FULL  = AugmentConfig(prob_hflip=0.5, brightness=(0.9, 1.1),     gamma=(0.95, 1.05),   noise_std=0.005)
AUG_CFG       = AUG_CFG_LIGHT

# Modo seguro: baja carga para pruebas rápidas
if SAFE_MODE:
    NUM_WORKERS = 0
    PREFETCH    = None
    PIN_MEMORY  = False
    PERSISTENT  = False
    USE_OFFLINE_BALANCED = False
    USE_ONLINE_BALANCING = False
    AUG_CFG = None

print(f"[CFG] OFFLINE_SPIKES={USE_OFFLINE_SPIKES}  RUNTIME_ENCODE={RUNTIME_ENCODE}")
print(f"[CFG] OFFLINE_BALANCED={USE_OFFLINE_BALANCED}  ONLINE_BALANCING={USE_ONLINE_BALANCING}")
print(f"[CFG] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")


[CFG] OFFLINE_SPIKES=False  RUNTIME_ENCODE=True
[CFG] OFFLINE_BALANCED=True  ONLINE_BALANCING=False
[CFG] workers=12 prefetch=2 pin=True persistent=True


__Verificación de datos (splits y opcionalmente H5)__

In [3]:
# =============================================================================
# Verificación de datos (splits y, si procede, H5)
# =============================================================================
PROC = ROOT/"data"/"processed"
TASKS_FILE = PROC / ("tasks_balanced.json" if USE_OFFLINE_BALANCED else "tasks.json")
with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

print("Tareas y TRAIN CSV a usar:")
for t in task_list:
    print(f" - {t['name']}: {Path(t['paths']['train']).name}")

if USE_OFFLINE_BALANCED:
    missing = []
    for t in task_list:
        p = Path(t["paths"]["train"])
        if p.name != "train_balanced.csv" or not p.exists():
            missing.append(str(p))
    if missing:
        print("[WARN] Faltan balanceados:", missing, " → usando tasks.json (no balanceado).")
        USE_OFFLINE_BALANCED = False
        with open(PROC/"tasks.json", "r", encoding="utf-8") as f:
            tasks_json = json.load(f)
        task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]

print("OK: verificación de splits.")



Tareas y TRAIN CSV a usar:
 - circuito1: train_balanced.csv
 - circuito2: train_balanced.csv
OK: verificación de splits.


__Factory de DataLoaders (elige H5 o CSV automáticamente)__

In [4]:
# =============================================================================
# Factory de loaders (elige H5 offline o CSV + runtime encode)
# =============================================================================
from src.utils import build_make_loader_fn

make_loader_fn = build_make_loader_fn(
    root=ROOT,
    use_offline_spikes=USE_OFFLINE_SPIKES,
    runtime_encode=RUNTIME_ENCODE,   # nombre del parámetro en el builder
)
print("make_loader_fn listo.")


make_loader_fn listo.


__Modelo y helper de construcción__

In [5]:
# =============================================================================
# Construcción del modelo (factory)
# =============================================================================
def make_model_fn(tfm):
    """
    Devuelve el modelo con los hyperparámetros de neuronas (beta/threshold).
    Para 'pilotnet_snn' estos kwargs aplican; para otros modelos se ignoran.
    """
    return build_model(MODEL_NAME, tfm, beta=0.9, threshold=0.5)

print("Modelo:", MODEL_NAME)


Modelo: pilotnet_snn


__Parche con métrica it/s por época (Opcional)__

In [6]:
# =============================================================================
# (Opcional) Parche: imprime it/s por época durante el entrenamiento
# =============================================================================
import time, json
from pathlib import Path
import torch
from torch import nn, optim
from torch.amp import autocast, GradScaler

import src.training as training
from src.utils import set_seeds

orig_train_supervised = training.train_supervised  # backup

def train_supervised_ips(model: nn.Module, train_loader, val_loader, loss_fn: nn.Module,
                         cfg, out_dir: Path, method=None):
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    if cfg.seed is not None:
        set_seeds(cfg.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    opt = optim.Adam(model.parameters(), lr=cfg.lr)
    use_amp = bool(cfg.amp and torch.cuda.is_available())
    scaler = GradScaler(enabled=use_amp)

    history = {"train_loss": [], "val_loss": []}
    for epoch in range(1, cfg.epochs + 1):
        # -------- train --------
        model.train()
        running = 0.0; nb = 0
        t0 = time.perf_counter()

        for x, y in train_loader:
            # _permute_if_needed: mantiene la convención (T,C,H,W) según el modelo
            x = training._permute_if_needed(x).to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)

            opt.zero_grad(set_to_none=True)
            with autocast("cuda", enabled=use_amp):
                y_hat = model(x)
                loss = loss_fn(y_hat, y)
                if method is not None:
                    loss = loss + method.penalty()

            if use_amp:
                scaler.scale(loss).backward()
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(opt); scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                opt.step()

            running += loss.item(); nb += 1

        dt = time.perf_counter() - t0
        ips = nb / dt if dt > 0 else float("nan")
        print(f"[TRAIN it/s] epoch {epoch}/{cfg.epochs}: {ips:.1f} it/s  ({nb} iters en {dt:.2f}s)")

        train_loss = running / max(1, nb)

        # -------- val --------
        model.eval()
        v_running = 0.0; nvb = 0
        with torch.no_grad():
            for x, y in val_loader:
                x = training._permute_if_needed(x).to(device, non_blocking=True)
                y = y.to(device, non_blocking=True)
                with autocast("cuda", enabled=use_amp):
                    y_hat = model(x)
                    v_loss = loss_fn(y_hat, y)
                v_running += v_loss.item(); nvb += 1

        val_loss = v_running / max(1, nvb)
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)

    manifest = {
        "epochs": cfg.epochs, "batch_size": cfg.batch_size, "lr": cfg.lr,
        "amp": cfg.amp, "seed": cfg.seed, "history": history,
    }
    (out_dir / "manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    return history

training.train_supervised = train_supervised_ips
print("Parche it/s ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised")


Parche it/s ACTIVADO. Para desactivarlo: training.train_supervised = orig_train_supervised


__Grid de métodos/presets/semillas/encoders__

In [7]:
# =============================================================================
# Config de comparativas: métodos, presets, semillas y encoders
# =============================================================================
# Qué métodos lanzar (ajusta a tus pruebas)
METHOD_SPECS = {
    "naive": [{}],
    "ewc": [
        {"lam": 7e8, "fisher_batches": 800},   # ajusta con FISHER_BY_PRESET si quieres
    ],
    "rehearsal": [
        {"buffer_size": 5000, "replay_ratio": 0.2},
    ],
    "rehearsal+ewc": [
        {"buffer_size": 5000, "replay_ratio": 0.2, "lam": 7e8, "fisher_batches": 800},
    ],
}

SEEDS         = [42]          # o varias semillas
ENCODERS      = ["rate"]      # añade "latency" si te interesa
PRESETS_TO_RUN= ["fast"]      # "fast" | "std" | "accurate"

print("Métodos:", METHOD_SPECS)
print("Presets:", PRESETS_TO_RUN, "| Seeds:", SEEDS, "| Encoders:", ENCODERS)


Métodos: {'naive': [{}], 'ewc': [{'lam': 700000000.0, 'fisher_batches': 800}], 'rehearsal': [{'buffer_size': 5000, 'replay_ratio': 0.2}], 'rehearsal+ewc': [{'buffer_size': 5000, 'replay_ratio': 0.2, 'lam': 700000000.0, 'fisher_batches': 800}]}
Presets: ['fast'] | Seeds: [42] | Encoders: ['rate']


__Lanzador de experimentos (driver)__

In [ ]:
# =============================================================================
# Driver de ejecución: recorre encoders x seeds x presets x métodos
# =============================================================================
from src.runner import run_continual

for enc in ENCODERS:
    for seed in SEEDS:
        for preset_i in PRESETS_TO_RUN:
            for method, specs in METHOD_SPECS.items():
                for spec_ in specs:
                    print(f"\n=== RUN: preset={preset_i} | method={method} | seed={seed} | enc={enc} | kwargs={spec_} ===")
                    try:
                        out_path, _ = run_continual(
                            task_list=task_list,
                            make_loader_fn=make_loader_fn,
                            make_model_fn=make_model_fn,
                            tfm=tfm,
                            preset=preset_i,
                            method=method,
                            seed=seed,
                            encoder=enc,
                            runtime_encode=RUNTIME_ENCODE,   # nombre unificado
                            out_root=ROOT/"outputs",
                            verbose=True,
                            method_kwargs=spec_,             # único sitio con kwargs del método
                        )
                        print("OK:", out_path)
                    except FileNotFoundError as e:
                        print("[ERROR] Falta algún H5 o split:", e)
                        print("Sugerencia: genera H5 con tools/encode_tasks.py o desactiva USE_OFFLINE_SPIKES.")

print("\nListo. Ejecuta las celdas de resumen ↓")



=== RUN: preset=fast | method=naive | seed=42 | enc=rate | kwargs={} ===

--- Tarea 1/2: circuito1 | preset=fast | method=naive | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)
[TRAIN it/s] epoch 1/2: 12.3 it/s  (909 iters en 73.84s)
[TRAIN it/s] epoch 2/2: 12.0 it/s  (909 iters en 75.84s)
  runtime encode: OFF

--- Tarea 2/2: circuito2 | preset=fast | method=naive | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)
[TRAIN it/s] epoch 1/2: 10.4 it/s  (202 iters en 19.48s)
[TRAIN it/s] epoch 2/2: 10.8 it/s  (202 iters en 18.77s)
  runtime encode: OFF
OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_naive_rate_model-PilotNetSNN_66x200_gray_seed_42

=== RUN: preset=fast | method=ewc | seed=42 | enc=rate | kwargs={'lam': 700000000.0, 'fisher_batches': 800} ===

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_7e+08 | B=64 T=10 AMP=True | enc=rate ---
  runtime encode: ON (GPU)


__Listado rápido de experimentos__

In [ ]:
# =============================================================================
# ¿Qué 'continual_*' hay en outputs?
# =============================================================================
cand = sorted((ROOT/"outputs").glob("continual_*"))
print("Encontrados:", len(cand))
for p in cand:
    print(" -", p.name, "| results.json:", (p/"continual_results.json").exists())


Encontrados: 7
 - continual_fast_ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_7e+08_lam_7e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_naive_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_naive_rate_model-SNNVisionRegressor_80x160_gray_seed_42 | results.json: True
 - continual_fast_rehearsal_buf_5000_rr_20+ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_rehearsal_buf_5000_rr_20+ewc_lam_7e+08_lam_7e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_rehearsal_buf_5000_rr_20_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True


__Resumen comparativo (parseo de nombres)__

In [ ]:
# =============================================================================
# Resumen comparativo de resultados
# =============================================================================
import re, json
import pandas as pd
from IPython.display import display

ALLOWED_ENC = r"(rate|latency|raw|image)"

def parse_exp_name(name: str):
    """
    Formato esperado:
      continual_<preset>_<tag>_<encoder>[_model-<model>]?_seed_<seed>?
    Ejemplos de <tag>:
      naive, ewc_lam_1e+09, rehearsal, rehearsal+ewc_lam_1e+09
    """
    pat = re.compile(
        rf"^continual_"
        rf"(?P<preset>[^_]+)_"
        rf"(?P<tag>.+)_"
        rf"(?P<enc>{ALLOWED_ENC})"
        rf"(?:_model\-(?P<model>.+?))?"
        rf"(?:_seed_(?P<seed>\d+))?$"
    )
    m = pat.match(name)
    meta = {"preset": None, "method": None, "lambda": None, "encoder": None, "seed": None, "model": None}
    if not m:
        return meta
    preset = m.group("preset"); tag = m.group("tag"); enc = m.group("enc")
    seed = m.group("seed"); model = m.group("model")
    lam = None; mlam = re.search(r"_lam_([^_]+)", tag)
    if mlam:
        lam = mlam.group(1)
        method = tag.replace(f"_lam_{lam}", "")
    else:
        method = tag
    return {"preset": preset, "method": method, "lambda": lam, "encoder": enc,
            "seed": int(seed) if seed is not None else None, "model": model}

rows = []
root_out = ROOT / "outputs"
for exp_dir in sorted(root_out.glob("continual_*")):
    name = exp_dir.name
    meta = parse_exp_name(name)
    if meta["preset"] is None:
        continue
    results_path = exp_dir / "continual_results.json"
    if not results_path.exists():
        continue
    with open(results_path, "r", encoding="utf-8") as f:
        res = json.load(f)

    task_names = list(res.keys())
    if len(task_names) < 2:
        continue  # con 1 tarea no hay after_*

    # heurística para encontrar primera y última tarea
    def is_last(d: dict) -> bool:
        return not any(k.startswith("after_") for k in d.keys())
    last_task = None; first_task = None
    for tn in task_names:
        if is_last(res[tn]): last_task = tn
        else: first_task = tn
    if first_task is None or last_task is None:
        task_names_sorted = sorted(task_names)
        first_task = task_names_sorted[0]; last_task = task_names_sorted[-1]

    c1, c2 = first_task, last_task
    c1_test_mae = float(res[c1].get("test_mae", float("nan")))
    c2_test_mae = float(res[c2].get("test_mae", float("nan")))
    after_key_mae = f"after_{c2}_mae"
    c1_after_c2_mae = float(res[c1].get(after_key_mae, float("nan")))
    forgetting_abs = c1_after_c2_mae - c1_test_mae
    forgetting_rel = (forgetting_abs / c1_test_mae * 100.0) if c1_test_mae == c1_test_mae else float("nan")

    rows.append({
        "exp": name, "preset": meta["preset"], "method": meta["method"], "lambda": meta["lambda"],
        "encoder": meta["encoder"], "model": meta["model"], "seed": meta["seed"],
        "c1_name": c1, "c2_name": c2,
        "c1_mae": c1_test_mae, "c1_after_c2_mae": c1_after_c2_mae,
        "c1_forgetting_mae_abs": forgetting_abs, "c1_forgetting_mae_rel_%": forgetting_rel,
        "c2_mae": c2_test_mae,
    })

df = pd.DataFrame(rows)
print(f"runs en resumen: {len(df)}")
if not df.empty:
    df["lambda_num"] = pd.to_numeric(df["lambda"], errors="coerce")
    df["seed"] = pd.to_numeric(df["seed"], errors="coerce").astype("Int64")
    df = df.sort_values(
        by=["preset", "method", "encoder", "model", "lambda_num", "seed"],
        na_position="last", ignore_index=True,
    )
    display(df[[ "exp","preset","method","lambda","encoder","model","seed",
                 "c1_name","c2_name","c1_mae","c1_after_c2_mae",
                 "c1_forgetting_mae_abs","c1_forgetting_mae_rel_%","c2_mae","lambda_num"]])
else:
    print("No hay filas (¿no existen JSONs o solo hubo 1 tarea por run?).")


runs en resumen: 7


,exp,preset,method,lambda,encoder,model,seed,c1_name,c2_name,c1_mae,c1_after_c2_mae,c1_forgetting_mae_abs,c1_forgetting_mae_rel_%,c2_mae,lambda_num
0,continual_fast_ewc_lam_7e+08_lam_7e+08_rate_mo...,fast,ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.160744,0.160831,0.000087,0.053922,0.242825,7.000000e+08
1,continual_fast_ewc_lam_1e+09_lam_1e+09_rate_mo...,fast,ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.149157,0.142538,-0.006620,-4.438050,0.248060,1.000000e+09
2,continual_fast_naive_rate_model-PilotNetSNN_66...,fast,naive,None,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.159975,0.186115,0.026141,16.340556,0.143623,NaN
3,continual_fast_naive_rate_model-SNNVisionRegre...,fast,naive,None,rate,SNNVisionRegressor_80x160_gray,42,circuito1,circuito2,0.177120,0.221733,0.044613,25.188146,0.177002,NaN
4,continual_fast_rehearsal_buf_5000_rr_20_rate_m...,fast,rehearsal_buf_5000_rr_20,None,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.159975,0.135997,-0.023978,-14.988568,0.149022,NaN
5,continual_fast_rehearsal_buf_5000_rr_20+ewc_la...,fast,rehearsal_buf_5000_rr_20+ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.160744,0.157076,-0.003668,-2.281876,0.233532,7.000000e+08
6,continual_fast_rehearsal_buf_5000_rr_20+ewc_la...,fast,rehearsal_buf_5000_rr_20+ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.149157,0.146060,-0.003097,-2.076296,0.225730,1.000000e+09


__Agregados y export a CSV__

In [ ]:
# =============================================================================
# Agregados y export a CSV
# =============================================================================
import pandas as pd
from IPython.display import display

if df.empty:
    print("df está vacío; salta agregados.")
else:
    cols_metrics = ["c1_mae", "c1_after_c2_mae", "c1_forgetting_mae_abs", "c1_forgetting_mae_rel_%", "c2_mae"]
    gdf = df.copy()
    if "lambda_num" not in gdf.columns:
        gdf["lambda_num"] = pd.to_numeric(gdf["lambda"], errors="coerce")

    agg = (
        gdf
        .groupby(["preset", "method", "encoder", "lambda", "lambda_num"], dropna=False)[cols_metrics]
        .agg(["mean", "std", "count"])
        .reset_index()
    )
    agg.columns = ["_".join(filter(None, map(str, col))).rstrip("_") for col in agg.columns.to_flat_index()]
    agg = agg.sort_values(by=["preset", "method", "encoder", "lambda_num"], na_position="last", ignore_index=True)

    summary_dir = ROOT / "outputs" / "summary"
    summary_dir.mkdir(parents=True, exist_ok=True)
    out_csv = summary_dir / "continual_summary_agg.csv"
    agg.to_csv(out_csv, index=False)
    print("Guardado:", out_csv)
    display(agg.head(20))


Guardado: /home/cesar/proyectos/TFM_SNN/outputs/summary/continual_summary_agg.csv


,preset,method,encoder,lambda,lambda_num,c1_mae_mean,c1_mae_std,c1_mae_count,c1_after_c2_mae_mean,c1_after_c2_mae_std,c1_after_c2_mae_count,c1_forgetting_mae_abs_mean,c1_forgetting_mae_abs_std,c1_forgetting_mae_abs_count,c1_forgetting_mae_rel_%_mean,c1_forgetting_mae_rel_%_std,c1_forgetting_mae_rel_%_count,c2_mae_mean,c2_mae_std,c2_mae_count
0,fast,ewc,rate,7e+08,7.000000e+08,0.160744,NaN,1,0.160831,NaN,1,0.000087,NaN,1,0.053922,NaN,1,0.242825,NaN,1
1,fast,ewc,rate,1e+09,1.000000e+09,0.149157,NaN,1,0.142538,NaN,1,-0.006620,NaN,1,-4.438050,NaN,1,0.248060,NaN,1
2,fast,naive,rate,NaN,NaN,0.168547,0.012124,2,0.203924,0.025186,2,0.035377,0.013062,2,20.764351,6.256191,2,0.160313,0.023602,2
3,fast,rehearsal_buf_5000_rr_20,rate,NaN,NaN,0.159975,NaN,1,0.135997,NaN,1,-0.023978,NaN,1,-14.988568,NaN,1,0.149022,NaN,1
4,fast,rehearsal_buf_5000_rr_20+ewc,rate,7e+08,7.000000e+08,0.160744,NaN,1,0.157076,NaN,1,-0.003668,NaN,1,-2.281876,NaN,1,0.233532,NaN,1
5,fast,rehearsal_buf_5000_rr_20+ewc,rate,1e+09,1.000000e+09,0.149157,NaN,1,0.146060,NaN,1,-0.003097,NaN,1,-2.076296,NaN,1,0.225730,NaN,1


__Tabla formateada “bonita”__

In [ ]:
# =============================================================================
# Tabla formateada (opcional)
# =============================================================================
from IPython.display import display

if df.empty or 'agg' not in globals():
    print("No hay datos agregados; salta tabla formateada.")
else:
    def fmt(x, prec=4):
        import pandas as pd
        return "" if pd.isna(x) else f"{x:.{prec}f}"

    show = agg.copy()
    count_cols = [c for c in show.columns if c.endswith("_count")]
    if count_cols:
        show["count"] = show[count_cols[0]].astype("Int64")
        show = show.drop(columns=count_cols)

    for c in [c for c in show.columns if c.endswith("_mean") or c.endswith("_std")]:
        show[c] = show[c].map(lambda v: fmt(v, 4))

    cols = [
        "preset", "method", "encoder", "lambda",
        "c1_mae_mean", "c1_forgetting_mae_rel_%_mean", "c2_mae_mean",
        "c1_mae_std",  "c1_forgetting_mae_rel_%_std",  "c2_mae_std",
        "count"
    ]
    cols = [c for c in cols if c in show.columns]  # por si falta alguna
    show = show[cols].rename(columns={
        "preset": "preset",
        "method": "método",
        "encoder": "codificador",
        "lambda": "λ",
        "c1_mae_mean": "MAE Tarea1 (media)",
        "c1_forgetting_mae_rel_%_mean": "Olvido T1 (%) (media)",
        "c2_mae_mean": "MAE Tarea2 (media)",
        "c1_mae_std": "MAE Tarea1 (σ)",
        "c1_forgetting_mae_rel_%_std": "Olvido T1 (%) (σ)",
        "c2_mae_std": "MAE Tarea2 (σ)",
        "count": "n (semillas)"
    })
    display(show)


,preset,método,codificador,λ,MAE Tarea1 (media),Olvido T1 (%) (media),MAE Tarea2 (media),MAE Tarea1 (σ),Olvido T1 (%) (σ),MAE Tarea2 (σ),n (semillas)
0,fast,ewc,rate,7e+08,0.1607,0.0539,0.2428,,,,1
1,fast,ewc,rate,1e+09,0.1492,-4.4380,0.2481,,,,1
2,fast,naive,rate,NaN,0.1685,20.7644,0.1603,0.0121,6.2562,0.0236,2
3,fast,rehearsal_buf_5000_rr_20,rate,NaN,0.1600,-14.9886,0.1490,,,,1
4,fast,rehearsal_buf_5000_rr_20+ewc,rate,7e+08,0.1607,-2.2819,0.2335,,,,1
5,fast,rehearsal_buf_5000_rr_20+ewc,rate,1e+09,0.1492,-2.0763,0.2257,,,,1
